In [1]:
import csv, copy
from collections import defaultdict
import numpy, re, math
from sklearn.linear_model import Ridge, LinearRegression
from string import punctuation
from scipy import stats
import pandas
from collections import defaultdict
from sklearn import linear_model, datasets
from collections import Counter
import random
from sklearn import svm
import matplotlib.pyplot as plt

In [2]:
data = pandas.read_csv("C:\Users\Induja\Documents\CSE 258\Assignment 2\h1b_kaggle.csv")
data = data.set_index('Unnamed: 0')
# Removing rows with NA in case_status
data = data[pandas.notnull(data['CASE_STATUS'])]
data = data.drop(data[data.CASE_STATUS == 'CERTIFIED-WITHDRAWN'].index)
data = data.drop(data[data.CASE_STATUS == 'INVALIDATED'].index)
data = data.drop(data[data.CASE_STATUS == 'WITHDRAWN'].index)
data = data.drop(data[data.CASE_STATUS == 'PENDING QUALITY AND COMPLIANCE REVIEW - UNASSIGNED'].index)

# To keep rows which have employer names that are among the ~1200 most frequent
#threshold = 0.0001
#for col in df:
#col = 'EMPLOYER_NAME'
#counts = data[col].value_counts(normalize=True)
#data = data.loc[data[col].isin(counts[counts > threshold].index), :]
data = data[data.CASE_STATUS != 'NA']
data = data[data.FULL_TIME_POSITION != 'NA']

# Case_status = 1 for certified/certified-withdrawn, 0 for denied
data['CASE_STATUS'] = data['CASE_STATUS'].map({'CERTIFIED': 1, 'DENIED': 0, 'REJECTED': 2})
data['CASE_STATUS'].replace(2,0, inplace=True)
data['FULL_TIME_POSITION'] = data['FULL_TIME_POSITION'].map({'Y': 1, 'N': 0})

# Separating states and adding that as another column
states = []
for row in data.iterrows():
    st = row[1]['WORKSITE'].split(',')[-1][1:]
    states.append(st)
s = pandas.Series(states)
data['STATES'] = s.values

data = data[data.STATES != 'NA']
data = data[data.FULL_TIME_POSITION.notnull()]
data = data[data.PREVAILING_WAGE.notnull()]

In [3]:
neg = data[data.CASE_STATUS == 0]
pos = data[data.CASE_STATUS == 1]
pos = pos.sample(frac=0.04).reset_index(drop=True)  # Random sampling from +ve data, to keep 0.04 times the original
frames = [pos, neg]
result = pandas.concat(frames)
result = result.sample(frac=1).reset_index(drop=True)  # Shuffling result
result = result[result.PREVAILING_WAGE.notnull()]
# Making buckets for wages, boundaries given in bins=[]
result['wage_bucket'] = pandas.cut(result['PREVAILING_WAGE'], bins=[0, 50000, 60000, 70000, 80000, 90000, 100000, 110000, 120000, 130000, 140000, 150000, 200000, 1000000000], labels=False)
result = result.dropna(subset=['wage_bucket'])
result.wage_bucket = result.wage_bucket.astype(int)
result.YEAR = result.YEAR.astype(int)

In [5]:
state_map = {
    'ALABAMA': 1,
    'ALASKA': 2,
    'ARIZONA': 3,
    'ARKANSAS': 4,
    'CALIFORNIA': 5,
    'COLORADO': 6,
    'CONNECTICUT': 7,
    'DELAWARE': 8,
    'DISTRICT OF COLUMBIA': 9,
    'FLORIDA': 10,
    'GEORGIA': 11,
    'HAWAII': 12,
    'IDAHO': 13,
    'ILLINOIS': 14,
    'INDIANA': 15,
    'IOWA': 16,
    'KANSAS': 17,
    'KENTUCKY': 18,
    'LOUISIANA': 19,
    'MAINE': 20,
    'MARYLAND': 21,
    'MASSACHUSETTS': 22,
    'MICHIGAN': 23,
    'MINNESOTA': 24,
    'MISSISSIPPI': 25,
    'MISSOURI': 26,
    'MONTANA': 27,
    'NEBRASKA': 28,
    'NEVADA': 29,
    'NEW HAMPSHIRE': 30,
    'NEW JERSEY': 31,
    'NEW MEXICO': 32,
    'NEW YORK': 33,
    'NORTH CAROLINA': 34,
    'NORTH DAKOTA': 35,
    'OHIO': 36,
    'OKLAHOMA': 37,
    'OREGON': 38,
    'PENNSYLVANIA': 39,
    'PUERTO RICO': 40,
    'RHODE ISLAND': 41,
    'SOUTH CAROLINA': 42,
    'SOUTH DAKOTA': 43,
    'TENNESSEE': 44,
    'TEXAS': 45,
    'UTAH': 46,
    'VERMONT': 47,
    'VIRGINIA': 48,
    'WASHINGTON': 49,
    'WEST VIRGINIA': 50,
    'WISCONSIN': 51,
    'WYOMING': 52}

def classifierStats(pred, y):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    for i in range(len(pred)):
        if y[i] == 1:
            if pred[i] == 1:
                tp += 1
            else:
                fn += 1
        else:
            if pred[i] == 1:
                fp += 1
            else:
                tn += 1
    FPR = 1.0*fp/(tn+fp)
    FNR = 1.0*fn/(tp+fn)
    print("Accuracy = " + str(1.0*(tp + tn)/(tn+fp+fn+tp)))
    print("FPR = " + str(FPR))
    print("FNR = " + str(FNR))
    print("BER = " + str(0.5*(FPR+FNR)))

In [6]:
wage_buck_num = len(result.wage_bucket.unique())
employer_freq = result.groupby('EMPLOYER_NAME').count().CASE_STATUS
employer_accept = pos.groupby('EMPLOYER_NAME').count().CASE_STATUS 
soc_freq = result.groupby('SOC_NAME').count().CASE_STATUS
soc_accept = pos.groupby('SOC_NAME').count().CASE_STATUS     
def feature(datum):
    feat = [0]*(1+6+52+wage_buck_num+2+2+1)
    feat[0] = 1
    feat[datum['YEAR'] - 2010] = 1
    feat[state_map[datum['STATES']]+6] = 1
    #feat[employers_id[datum['EMPLOYER_NAME']]+len(state_map)+6] = 1
    feat[datum['wage_bucket']+6+52] = 1
    feat[-1] = datum['FULL_TIME_POSITION']
    e = datum['EMPLOYER_NAME']
    if e in employer_freq:
        feat[-2] = employer_freq[e]
        if e in employer_accept:
            feat[-3] = employer_accept[e] * 1.0 / feat[-2]
    soc = datum['SOC_NAME']
    if soc in soc_freq:
        feat[-4] = soc_freq[soc]
        if soc in soc_accept:
            feat[-5] = soc_accept[soc] * 1.0 / feat[-4]
    return feat

X_train = [feature(d[1]) for d in result.loc[:len(result)/2,:].iterrows()]
y_train = [d[1]['CASE_STATUS'] for d in result.loc[:len(result)/2,:].iterrows()]

In [7]:
X_test =  [feature(d[1]) for d in result.loc[len(result)/2:,:].iterrows()]
y_test =  [d[1]['CASE_STATUS'] for d in result.loc[len(result)/2:,:].iterrows()]

In [8]:
logreg = linear_model.LogisticRegression(C=10)
logreg.fit(X_train, y_train)
pred = logreg.predict(X_test)
classifierStats(pred, y_test)

Accuracy = 0.855586518851
FPR = 0.214814656736
FNR = 0.0811950251998
BER = 0.148004840968


In [9]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 500, max_depth =40, max_features=10)
rf.fit(X_train, y_train)
pred = rf.predict(X_test)
classifierStats(pred, y_test)

Accuracy = 0.850750186789
FPR = 0.199278687125
FNR = 0.104325163367
BER = 0.151801925246


In [ ]:
clf = svm.SVC(C=10, kernel='rbf')
clf.fit(X_train, y_train)

train_predictions = clf.predict(X_train)
test_predictions = clf.predict(X_test)
classifierStats(test_predictions, y_test)

In [13]:
#Baseline 1 : ZeroR
y_consolidated = y_train + y_test 

from collections import Counter

def Most_Common(y_consolidated):
    data = Counter(y_consolidated)
    return data.most_common(1)[0][0]

pred_baselise1 = [Most_Common(y_consolidated)] * len(y_test)

#Accuracy & FNR & FPR
classifierStats(pred_baselise1, y_test)

Accuracy = 0.526877486319
FPR = 1.0
FNR = 0.0
BER = 0.5


In [15]:
import random
#Baseline 2 : Biased Coin Toss
sum = numpy.sum(y_consolidated)
sum = sum*1.0/len(y_consolidated)
#Accuracy
pred_baselise2 = [ (random.random() < sum) for x in range(len(y_test)) ]
#FNR & FPR
classifierStats(pred_baselise2, y_test)

Accuracy = 0.500676480685
FPR = 0.525726114514
FNR = 0.475614663779
BER = 0.500670389147
